In [1]:
import numpy as np
import os

from sklearn import metrics, cluster, preprocessing
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import pandas as pd
from itertools import cycle, islice

import seaborn as sns
import plotly as py
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

#### Normalization 
The following cell imports the dataset being clustered, converts the dataset from a data frame to a numpy array, and then normalizes the data using the RobustScalar provided by SciKit Learn.

In [2]:
path = "../data"

pulsar_data = pd.read_csv(path + "/input/HTRU2/HTRU_2.csv", 
                          low_memory=False, index_col=False, usecols=[*range(0, 7)], header=0, 
                          names=["mean_IP", "standDev_IP", "excessKurt_IP", "skewness_IP", 
                                 "mean_DMSNR", "standDev_DMSNR", "excessKurt_DMSNR", "skewness_DMSNR"])
ground_truth = pd.read_csv(path + "/input/HTRU2/HTRU_2.csv", 
                           low_memory=False, index_col=False, usecols=[8], header=0, 
                           names=["pulsar"])
X = pulsar_data.to_numpy()

X_tr = preprocessing.RobustScaler().fit_transform(X)
X_ts = preprocessing.StandardScaler().fit_transform(X)
X_tp = preprocessing.PowerTransformer().fit_transform(X)
X_tq = preprocessing.QuantileTransformer(output_distribution='normal').fit_transform(X)


#print(pulsar_data.head())
#print(ground_truth.head())
#print(ground_truth.tail())
#print(X)
#print(X_tr)

#### Clustering
This segment of code clusters the normalized data using optics and gets an array of labels for each instance that was clustered. A ground truth matrix of the same data is generated using the ground truth values from the original data.

In this case, eps=1 and min_samples=100 were chosen values for clustering because they form one large cluster and the rest is labeled as noise. This turns out to be useful because most of the true pulsars are identified as noise while most of the instances clustered were proven to be radiowaves and thus nonpulsars.

In [3]:
op_raw = cluster.OPTICS(eps=1, min_samples=100, cluster_method="dbscan").fit(X)
op_s = cluster.OPTICS(eps=1, min_samples=100, cluster_method="dbscan").fit(X_ts)
op_r = cluster.OPTICS(eps=1, min_samples=100, cluster_method="dbscan").fit(X_tr)
op_p = cluster.OPTICS(eps=1, min_samples=100, cluster_method="dbscan").fit(X_tp)
op_q = cluster.OPTICS(eps=1, min_samples=100, cluster_method="dbscan").fit(X_tq)

op = cluster.OPTICS(eps=1, min_samples=100, cluster_method="dbscan").fit(X_tr)

opclus = op.labels_
gt = ground_truth.to_numpy().transpose()[0]
print("Clusters formed: " + str(np.unique(op.labels_)))

Clusters formed: [-1  0]


#### Analysis
In this cell, a comparison of the ground truth values for the pulsars was compared to the clustering results. In this case, the data demonstrated the following patterns:

True Positive: Pulsars that were correctly identified as noise points by the clustering algorithm
False Negative: Pulsars that were clustered as part of the large main cluster despite being pulsars
False Positive: Nonpulsars/radio waves that were labeled as noise points and thus expected to be pulsars
True Negative: Nonpulsars/radio waves that were clustered into the large single cluster (not outliers)

In [4]:
#Intialize counts to find totals and TP, TN, FP, FN values
totalpulsars = 0
pulsarsfound = 0
totalnonpulsars = 0
radionoise = 0
radioclus = 0
pulsar_notmatch = []

numel = len(gt)
for i in range(numel):
    if gt[i] == 1: #Get pulsars
        totalpulsars += 1
        if opclus[i] == -1:
            pulsarsfound += 1 #Count pulsars found to be noise points (expected)
        elif opclus[i] == 0:
            pulsar_notmatch.append(i) #Document pulsars that were not noise points (unexpected)
    else:
        totalnonpulsars += 1 #Get nonpulsars
        if opclus[i] == -1: #Check for radiowaves identified as noise (FP)
            radionoise += 1
        elif opclus[i] == 0: #Check for true negative (Clustered radio waves)
            radioclus += 1
            

print("Total Pulsars: " + str(totalpulsars))
print("Total Nonpulsars: " + str(totalnonpulsars))
print("Pulsars identified as noise (TP): " + str(pulsarsfound))
print("Pulsars clustered (FN): " + str(len(pulsar_notmatch)))
print("Radio Waves identified as noise (FP): " + str(radionoise))
print("Radio Waves clustered (TN): " + str(radioclus))
print("\nPercent of pulsars correctly identified: " + str(100*pulsarsfound/totalpulsars))
print("\n\nPulsars not identified as puslars (FN): " + str(pulsar_notmatch))

Total Pulsars: 1639
Total Nonpulsars: 16258
Pulsars identified as noise (TP): 1460
Pulsars clustered (FN): 179
Radio Waves identified as noise (FP): 1465
Radio Waves clustered (TN): 14793

Percent of pulsars correctly identified: 89.07870652837096


Pulsars not identified as puslars (FN): [41, 253, 277, 302, 389, 408, 447, 462, 569, 595, 1029, 1286, 1463, 1493, 1496, 1500, 1527, 1592, 1603, 1617, 1624, 1736, 1786, 1888, 1908, 1930, 2091, 2218, 2229, 2251, 2304, 2306, 2424, 2619, 2624, 2672, 2797, 2855, 2942, 2976, 2987, 3029, 3108, 3113, 3134, 3135, 3140, 3141, 3193, 3217, 3228, 3253, 3258, 3329, 3336, 3423, 3424, 3427, 3431, 3433, 3435, 3497, 3549, 3573, 3574, 3600, 3608, 3620, 3639, 3651, 3662, 3740, 3793, 3833, 3933, 4020, 4023, 4042, 4067, 4083, 4182, 4204, 4230, 4448, 4597, 4663, 4687, 4789, 4797, 5005, 5046, 5113, 5122, 5132, 5141, 5142, 5155, 5285, 5589, 5600, 5628, 5646, 5647, 5796, 6123, 6659, 6712, 6920, 7007, 7192, 7228, 7276, 7286, 7313, 7485, 7497, 7520, 7542, 7570, 7659, 

#### Analysis
This cell contains the data for the rand index, adjusted mutual info, and fowlkes-mallowscoring of the clustering.

In [9]:
labels_truth = ground_truth.to_numpy().transpose()[0]
print(np.unique(labels_truth, return_counts=True))
print(np.unique(np.less(op_s.labels_, labels_truth), return_counts=True))
print(np.unique(np.less(op_r.labels_, labels_truth), return_counts=True))
print(np.unique(np.less(op_p.labels_, labels_truth), return_counts=True))
print(np.unique(np.less(op_q.labels_, labels_truth), return_counts=True))
op_s.labels_[op_s.labels_ == -1 ] = 1
op_r.labels_[op_r.labels_ == -1 ] = 1
op_p.labels_[op_s.labels_ == -1 ] = 1
op_q.labels_[op_r.labels_ == -1 ] = 1
print('OPTICS Clustering Metrics:')
labels_pred = op_s.labels_
print('z-score\\')
rand_s = metrics.rand_score(labels_truth, labels_pred)
print('rand score = ', rand_s)
amis_s = metrics.adjusted_mutual_info_score(labels_truth, labels_pred) 
print('adjusted mutual info score = ', amis_s)
fm_s = metrics.fowlkes_mallows_score(labels_truth, labels_pred)
print('fowlkes-mallow = ', fm_s)
labels_pred = op_r.labels_
print('robust_scalar\\')
rand_r = metrics.rand_score(labels_truth, labels_pred)
print('rand score = ', rand_r)
amis_r = metrics.adjusted_mutual_info_score(labels_truth, labels_pred)
print('adjusted mutual info score = ', amis_r)
fm_r = metrics.fowlkes_mallows_score(labels_truth, labels_pred)
print('fowlkes-mallow = ', fm_r)
labels_pred = op_p.labels_
print('power_scalar\\')
rand_p = metrics.rand_score(labels_truth, labels_pred)
print('rand score = ', rand_p)
amis_p = metrics.adjusted_mutual_info_score(labels_truth, labels_pred)
print('adjusted mutual info score = ', amis_p)
fm_p = metrics.fowlkes_mallows_score(labels_truth, labels_pred)
print('fowlkes-mallow = ', fm_p)
labels_pred = op_q.labels_
print('quantile_scalar\\')
rand_q = metrics.rand_score(labels_truth, labels_pred)
print('rand score = ', rand_q)
amis_q = metrics.adjusted_mutual_info_score(labels_truth, labels_pred)
print('adjusted mutual info score = ', amis_q)
fm_q = metrics.fowlkes_mallows_score(labels_truth, labels_pred)
print('fowlkes-mallow = ', fm_q)

(array([0, 1]), array([16258,  1639]))
(array([False,  True]), array([17363,   534]))
(array([False,  True]), array([17718,   179]))
(array([False,  True]), array([15673,  2224]))
(array([False,  True]), array([15397,  2500]))
OPTICS Clustering Metrics:
z-score\
rand score =  0.8806149948237304
adjusted mutual info score =  0.36302842491344656
fowlkes-mallow =  0.928113823461954
robust_scalar\
rand score =  0.8331488766157531
adjusted mutual info score =  0.36902371295939224
fowlkes-mallow =  0.8951643434856997
power_scalar\
rand score =  0.7873156056227872
adjusted mutual info score =  -0.00012153251345185578
fowlkes-mallow =  0.880778556838967
quantile_scalar\
rand score =  0.7671794587560583
adjusted mutual info score =  -6.378850245011445e-05
fowlkes-mallow =  0.8662476762308446


In [13]:
OPTICS_Metrics = pd.DataFrame([["OPTICS", "z-score", rand_s, fm_s], ["OPTICS", "robust scalar", rand_r, fm_r]], columns=["Clustering Algorithm", "Normalization Method", 
                                           "Rand Index", "Fowlkes-Mallow"])

OPTICS_Metrics.to_csv(path + "/output/optics_metrics.csv")
OPTICS_Metrics

,Clustering Algorithm,Normalization Method,Rand Index,Fowlkes-Mallow
0,OPTICS,z-score,0.880615,0.928114
1,OPTICS,robust scalar,0.833149,0.895164


#### Identifying Pulsars:
This code has been commented out, but it can be used to confirm that a majority of the pulsars found using the ground truth are noise points and a majority of the nonpulsars are part of the single large cluster.

In [ ]:
"""
numel = len(eq_matrix)
count = 0
zeros = 0
pulsars = []
clust1 = []
clust2 = []
for i in range(numel):
    if eq_matrix[i] == True:
        count += 1
        if gt[i] == 0:
            zeros += 1
    if gt[i] == 1:
        pulsars.append(i)
    if opclus[i] == -1:
        clust2.append(i)
    elif opclus[i] == 1:
        clust1.append(i)
        
print(count)
print(count/gt.size)
print(zeros/gt.size) #matching zeros (non-pulsars)
print(pulsars)
print(clust2)
print(clust1)
"""